In [1]:
import mido

In [6]:
from mido import MidiFile
from mido import Message

In [3]:
def download_midi(count):
    list_mid = []
    for i in range(1, count + 1):
        path = '/Users/diana/Documents/Project/train_songs_3/' + str(i) + '.MID'
        list_mid.append(MidiFile(path))
    return list_mid

In [4]:
def get_data_from_midi(mid, dur):
    note = []
    msg_type = []
    time = []
    time_now = 0
    for msg in mid.play():
        time_now += msg.time
        if (time_now > dur):
            return note, msg_type, time
        if (msg.type == 'note_on' or msg.type == 'note_off'):
            msg_type.append(msg.type)
            note.append(msg.note)
            time.append(time_now)
    print(time_now)
    return note, msg_type, time

In [5]:
def resize_data(note, msg_type, time, sz):
    note = note[:sz]
    msg_type = msg_type[:sz]
    time = time[:sz]
    return note, msg_type, time

In [6]:
def type_dur(time, max_dur_note, amount_dur):
    for i in range(amount_dur):
        if (max_dur_note / amount_dur * i < time and max_dur_note / amount_dur * (i + 1) >= time):
            return i
    return amount_dur - 1

In [7]:
def indexes_from_midi(note, msg_type, time, max_dur_note, amount_dur, each_dur):
    indexes = []
    time_last = 0
    empty = 128 * amount_dur
    sz = len(note)
    
    for i in range(sz):
        if (msg_type[i] == 'note_on'):
            j = i + 1
            while (j < sz and note[i] != note[j]):
                j += 1
            if (j >= sz):
                t_dur = type_dur(max_dur_note, max_dur_note, amount_dur)
            else:
                t_dur = type_dur(time[j] - time[i], max_dur_note, amount_dur)
                print(t_dur)
            while (time_last + each_dur < time[i]):
                indexes.append(empty)
                time_last += each_dur
            indexes.append(note[i] + 128 * t_dur)
    indexes.append(EOS_token)
    return indexes

In [8]:
from mido import Message, MidiFile, MidiTrack
from operator import itemgetter
def midi_from_indexes(indexes, max_dur_note, amount_dur, each_dur):
    time_now = 0
    events = []
    for i in range(len(indexes)):
        if (indexes[i] == '<EOS>'):
            note = -1
        elif (indexes[i] == 128 * amount_dur):
            note = -1
        else:
            note = indexes[i] % 128
            
            type_dur = (indexes[i] - note) / 128
            print(type_dur)
            dur = max_dur_note / amount_dur * (type_dur + 1)
        if (note != -1):
            events.append([time_now, 'note_on', note])
            events.append([time_now + dur, 'note_off', note])
        time_now += each_dur
    events = sorted(events,key=itemgetter(0))
    for i in range(len(events) - 1, 0, -1):
        events[i][0] -= events[i - 1][0]
    mid = MidiFile()
    track = MidiTrack()
    mid.tracks.append(track)
    for ev in events:
        track.append(Message(ev[1], note=ev[2], velocity=64, time=int(ev[0]/0.0013354687499999999)))
    return mid

In [9]:
def get_list_data(dur, count):
    list_mid = download_midi(count)
    data = []
    for i in range(count):
        print(list_mid[i])
        note, msg_type, time = get_data_from_midi(list_mid[i], dur)
        data.append([note, msg_type, time])
    return data

In [12]:
max_dur_note = 4
amount_dur = 10
each_dur = 0.2
hidden_size = 256
list_indexes = []
duration = 60
count = 16
SOS_token = 128 * amount_dur + 1
EOS_token = 128 * amount_dur + 2

In [866]:
data = get_list_data(duration, count)

<midi file '/Users/diana/Documents/Project/train_songs_3/1.MID' type 0, 1 tracks, 5632 messages>
<midi file '/Users/diana/Documents/Project/train_songs_3/2.MID' type 0, 1 tracks, 8575 messages>
<midi file '/Users/diana/Documents/Project/train_songs_3/3.MID' type 0, 1 tracks, 11928 messages>
<midi file '/Users/diana/Documents/Project/train_songs_3/4.MID' type 0, 1 tracks, 15227 messages>
<midi file '/Users/diana/Documents/Project/train_songs_3/5.MID' type 0, 1 tracks, 19976 messages>
<midi file '/Users/diana/Documents/Project/train_songs_3/6.MID' type 0, 1 tracks, 20010 messages>
<midi file '/Users/diana/Documents/Project/train_songs_3/7.MID' type 0, 1 tracks, 24237 messages>
<midi file '/Users/diana/Documents/Project/train_songs_3/8.MID' type 0, 1 tracks, 24842 messages>
<midi file '/Users/diana/Documents/Project/train_songs_3/9.MID' type 0, 1 tracks, 25946 messages>
<midi file '/Users/diana/Documents/Project/train_songs_3/10.MID' type 0, 1 tracks, 34908 messages>
<midi file '/Users/di

In [11]:

for i in range(count):
    list_indexes.append(indexes_from_midi(data[i][0], data[i][1], data[i][2], max_dur_note, amount_dur, each_dur))

NameError: name 'data' is not defined

In [13]:
import random
import timeit
import time

In [14]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MAX_LENGTH = 100000000
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [15]:
teacher_forcing_ratio = 1


def train(input_tensor, target_tensor, decoder, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    
    loss = 0

    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = input_tensor
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    
    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()
    decoder_optimizer.step()

    return loss.item() / target_length

In [16]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [17]:
def trainIters(count, target_indexes, decoder, print_every=2, plot_every=100, learning_rate=0.000001, n_iters = 100, rand = 30):
    start = time.time()
    
    print_loss_total = 0  # Reset every print_every

    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    #decoder_optimizer = optim.RMSprop(decoder.parameters(), lr=0.01, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0, centered=False)
    #encoder_optimizer = torch.optim.ASGD(encoder.parameters(), lr=0.01, lambd=0.0001, alpha=0.75, t0=1000000.0, weight_decay=0)
    #decoder_optimizer = torch.optim.ASGD(decoder.parameters(), lr=0.01, lambd=0.0001, alpha=0.75, t0=1000000.0, weight_decay=0)
    
     
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        input_rand = []
        a = random.randint(1, rand)
        for i in range(0, hidden_size):
            input_rand.append(a)
        input_rand = np.reshape(input_rand, (1, 1, hidden_size))
        
        input_tensor = torch.FloatTensor(input_rand)
        t = target_indexes[iter%count]
        target_tensor = torch.tensor(t, dtype=torch.long, device=device).view(-1, 1)
        loss = train(input_tensor, target_tensor, 
                     decoder, decoder_optimizer, criterion)
        
        print_loss_total += loss
        
        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))
   

In [18]:
def evaluate(input_rand, decoder, max_length=MAX_LENGTH, most_prob = 5):
    with torch.no_grad():
        
        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS
        
        input_tensor = torch.FloatTensor(input_rand)
        decoder_hidden = input_tensor
        
        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(most_prob)
            i = random.randint(0, most_prob - 1)
            
            topv = topv[0][i]
            topi = topi[0][i]

            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(topi.item())
            decoder_input = topi.squeeze().detach()

        return decoded_words
  

In [19]:

decoder = DecoderRNN(hidden_size, 128 * amount_dur + 3).to(device)


In [938]:
for i in range(10):
    print(i)
    torch.save(decoder, "/Users/diana/Documents/Project/models_Sonya/decoder_" + str(5000 + i))
    trainIters(count, list_indexes, decoder, print_every=3, n_iters = 500)

0
0m 4s (- 12m 53s) (3 0%) 3.9383
0m 6s (- 9m 23s) (6 1%) 2.9825
0m 11s (- 10m 11s) (9 1%) 4.1634
0m 12s (- 8m 47s) (12 2%) 3.3511
0m 14s (- 7m 53s) (15 3%) 3.0942
0m 19s (- 8m 35s) (18 3%) 4.0531
0m 22s (- 8m 22s) (21 4%) 3.0392
0m 26s (- 8m 40s) (24 4%) 3.8413
0m 29s (- 8m 32s) (27 5%) 4.1267
0m 30s (- 7m 59s) (30 6%) 3.0516
0m 33s (- 7m 52s) (33 6%) 3.3249
0m 37s (- 8m 0s) (36 7%) 3.7215
0m 40s (- 7m 56s) (39 7%) 3.4878
0m 43s (- 7m 54s) (42 8%) 3.6302
0m 45s (- 7m 38s) (45 9%) 3.4217
0m 47s (- 7m 31s) (48 9%) 3.5790
0m 51s (- 7m 37s) (51 10%) 3.9307
0m 54s (- 7m 30s) (54 10%) 2.9820
0m 58s (- 7m 34s) (57 11%) 4.1488
1m 0s (- 7m 21s) (60 12%) 3.3053
1m 1s (- 7m 8s) (63 12%) 3.0280
1m 6s (- 7m 14s) (66 13%) 4.0516
1m 8s (- 7m 7s) (69 13%) 3.0441
1m 11s (- 7m 5s) (72 14%) 3.8608
1m 14s (- 7m 3s) (75 15%) 4.1056
1m 16s (- 6m 52s) (78 15%) 3.0928
1m 19s (- 6m 49s) (81 16%) 3.3240
1m 22s (- 6m 49s) (84 16%) 3.7082
1m 25s (- 6m 45s) (87 17%) 3.4995
1m 28s (- 6m 43s) (90 18%) 3.5984
1m 30s

3m 29s (- 4m 16s) (225 45%) 3.2690
3m 32s (- 4m 14s) (228 45%) 3.6576
3m 35s (- 4m 11s) (231 46%) 3.4392
3m 38s (- 4m 8s) (234 46%) 3.5285
3m 39s (- 4m 4s) (237 47%) 3.4215
3m 42s (- 4m 0s) (240 48%) 3.4971
3m 45s (- 3m 58s) (243 48%) 3.8609
3m 47s (- 3m 55s) (246 49%) 2.9526
3m 51s (- 3m 53s) (249 49%) 4.0802
3m 53s (- 3m 49s) (252 50%) 3.2518
3m 54s (- 3m 45s) (255 51%) 3.0767
3m 58s (- 3m 44s) (258 51%) 3.9806
4m 1s (- 3m 40s) (261 52%) 3.0043
4m 4s (- 3m 38s) (264 52%) 3.7990
4m 7s (- 3m 35s) (267 53%) 4.0173
4m 8s (- 3m 31s) (270 54%) 3.0241
4m 11s (- 3m 29s) (273 54%) 3.2669
4m 16s (- 3m 28s) (276 55%) 3.6454
4m 19s (- 3m 25s) (279 55%) 3.4311
4m 22s (- 3m 23s) (282 56%) 3.5552
4m 24s (- 3m 19s) (285 56%) 3.3889
4m 27s (- 3m 16s) (288 57%) 3.4222
4m 30s (- 3m 14s) (291 58%) 3.8589
4m 33s (- 3m 11s) (294 58%) 2.9723
4m 37s (- 3m 9s) (297 59%) 4.0745
4m 38s (- 3m 5s) (300 60%) 3.2847
4m 40s (- 3m 2s) (303 60%) 2.9826
4m 44s (- 3m 0s) (306 61%) 3.9724
4m 47s (- 2m 57s) (309 61%) 3.0

KeyboardInterrupt: 

# 